In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train = pd.read_csv('/content/gdrive/MyDrive/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/test.csv')
testOriginal = pd.read_csv('/content/gdrive/MyDrive/test.csv')

In [4]:
train.shape

(181507, 272)

In [5]:
test.drop('row ID', axis=1, inplace=True)

<h3>SubArea Removal<h3>

In [6]:
test.drop('sub_area', axis=1, inplace=True)
train.drop('sub_area', axis=1, inplace=True)

<h1>Categorical To Numerical<h1>

<h3>OneHot<h3>

In [7]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

<h1>Working<h1>

In [8]:
X = train.loc[:, train.columns != 'price_doc']
y = train[['price_doc']]

# scaler = RobustScaler()
# X = scaler.fit_transform(X)
# test = scaler.fit_transform(test)

In [9]:
X.shape

(181507, 287)

In [10]:
test.shape

(77789, 287)

<h1>P-Value Selection<h1>

In [11]:
threshold_value = 200000
variance_filter = VarianceThreshold(threshold=threshold_value)

X2 = variance_filter.fit_transform(X)
test2 = variance_filter.transform(test)

X2.shape
test2.shape

(77789, 47)

<h1> Poly Interaction ONNNNNNNNN<h1>

In [12]:
poly = PolynomialFeatures(2, interaction_only=True)
X3 = poly.fit_transform(X2)
test3 = poly.fit_transform(test2)




<h1>Applying Model<h1>

In [13]:
X3.shape

(181507, 1129)

In [14]:
test3.shape

(77789, 1129)

In [15]:
# Create and train the Lasso regression model
alpha_value = 0.5  # You can adjust the alpha parameter based on your needs
lasso_reg = Lasso(alpha=alpha_value, random_state=42)
lasso_reg.fit(X3, y)

# Print the coefficients and intercept
print("Coefficients:", lasso_reg.coef_)
print("Intercept:", lasso_reg.intercept_)

# Make predictions on the test data
y_pred_lasso = lasso_reg.predict(test3)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.547e+19, tolerance: 8.652e+15
  model = cd_fast.enet_coordinate_descent(


Coefficients: [ 0.00000000e+00  2.79862296e+04  1.30433856e+03 ... -1.43291074e-08
 -6.58555918e-05 -2.51221880e-04]
Intercept: [1607763.90025624]


In [16]:
# Save the predictions to a CSV file
result_df_lasso = pd.DataFrame({'row ID': testOriginal['row ID'], 'price_doc': y_pred_lasso.flatten()})
result_df_lasso.to_csv('/content/gdrive/MyDrive/Day_6_Abdullah_Maqsood_24448_Submission_6.csv', index=False)
